# Berlin Airbnb Data Analysis

In the following project I'm looking at data of Airbnb listings for the city of Berlin for the period of 07/2019 - 07/2020. 
The data was downloaded from the [insideairbnb.com](http://insideairbnb.com/get-the-data.html) website.

The goal of this notebook is to clean, transform and visualize the data in order to answer the three following questions:


1. Looking at the prices for Airbnb listings for Berlin, do prices differ among the Berlin neighborhoods? Do some neighborhoods have more expensive or cheaper Airbnb listings than others?

2. Do visitor numbers and listing prices fluctuate over the year?

3. How do Airbnb prices in Berlin compare to another large german city, Munich?

A short blog post discussing the results can be found [here](https://www.google.com).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 1. Looking at the prices for Airbnb listings for Berlin, do prices differ among the different Berlin neighborhoods?

In [ ]:
# Loading the listings data for Berlin
ber_listings_url = "http://data.insideairbnb.com/germany/be/berlin/2019-07-11/data/listings.csv.gz"
ber_listings = pd.read_csv(ber_listings_url)
ber_listings.head(3)

In [ ]:
# Loading the calendar data for Berlin
ber_calendar_url = "http://data.insideairbnb.com/germany/be/berlin/2019-07-11/data/calendar.csv.gz"
ber_calendar = pd.read_csv(ber_calendar_url)
ber_calendar.head(3)

In [ ]:
# Converting the price variable in ber_calendar to numeric
ber_calendar["price"] = pd.to_numeric(ber_calendar["price"].str.replace("$",""), errors="coerce")
ber_calendar = ber_calendar.dropna(subset=["price"])
                                   
# Calculating the average price for every listing throughout the whole year
ber_avg_price = ber_calendar.groupby("listing_id")["price"].mean()
ber_avg_price = ber_avg_price.reset_index()

# Adding the neighbourhood_group and number of beds information to ber_avg_price
ber_avg_price = pd.merge(ber_avg_price, ber_listings[["id", "neighbourhood_group_cleansed", "beds"]], left_on="listing_id", right_on="id")
ber_avg_price = ber_avg_price[["listing_id", "neighbourhood_group_cleansed", "price", "beds"]]

# Calculating average listing price for every Berlin district
ber_price_district = ber_avg_price.groupby("neighbourhood_group_cleansed")["price"].mean().sort_values(ascending=False)
ber_price_district = ber_price_district.reset_index()

In [ ]:
plt.figure(figsize=(8,5))

flierprops = dict(markersize=0.5, linestyle='none', color="#f4f4f4")

sort_median = ber_avg_price.groupby("neighbourhood_group_cleansed")["price"].median().sort_values(ascending=False)

sns.boxplot(ber_avg_price["price"], ber_avg_price["neighbourhood_group_cleansed"], 
            orient="h", flierprops=flierprops, order=sort_median.index, color="#0485d1")

# setting x_limit to 400, because of many outliers
plt.xlim(0, 400)

plt.ylabel("")
plt.xlabel("Listing Price ($)", size=12)
plt.title("Distribution of Prices in Berlin Districts", size=14, pad=12)

sns.despine()
plt.tight_layout();

All the different district in Berlin show a surprisingly wide range of listing prices. For instance, in Berlin-Pankow prices go from 9\\$ to 800\\$. This wide price range could be explained by the many factors that influence the listing price, for example room number, apartment size etc.
An additional explanation could be that the chosen districts (or neighborhood groups) are very large and diverse city areas comprised of very different smaller neighborhoods that might be more or less affluent.
The district with the highest median price of 59\\$ is Berlin-Pankow. Berlin-Reinickendorf on the other hand has only a median listing price of 40\\$.

One might get a better overview by looking at the 10% of the most expensive and least expensive listings repectively, and compare how they are distributed over the city:

In [ ]:
# Defining the cutoffs to select the 10% of listings with the highest and lowest prices respectively
high_cutoff = ber_avg_price["price"].quantile(0.90)
low_cutoff = ber_avg_price["price"].quantile(0.10)

expensive_listings = ber_avg_price[ber_avg_price["price"] >= high_cutoff]
cheap_listings = ber_avg_price[ber_avg_price["price"] <= low_cutoff]

# Selecting the 5% of listings with the highest prices
extreme_prices = expensive_listings.groupby("neighbourhood_group_cleansed")["price"].count()
extreme_prices = extreme_prices.reset_index()
extreme_prices = extreme_prices.rename(columns={"price":"Expensive"})

# Adding the 5% of listings with the lowest prices to extreme_prices
low_prices = cheap_listings.groupby("neighbourhood_group_cleansed")["price"].count()
low_prices = low_prices.reset_index()
extreme_prices["Cheap"] = low_prices["price"]

# Transforming extreme_prices for the visualization
extreme_prices = extreme_prices.sort_values("Expensive", ascending=False)
extreme_prices = pd.melt(extreme_prices, id_vars="neighbourhood_group_cleansed", var_name="Category", value_name="Listings")

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x='Listings', y='neighbourhood_group_cleansed', hue='Category', data=extreme_prices, 
            orient="h", palette=["#0485d1", "#ff474c"])

plt.ylabel("")
plt.xlabel("Number of Listings", size=12)
sns.despine()
plt.title("Number of Airbnb Listings in Berlin with Very High and Low Prices", size=14, pad=12)
plt.legend(frameon=False)
plt.tight_layout();

Looking at the distribution of 10% of the most and least expensive listings one can make a couple of interesting observations. Berlin-Mitte, the district in the center of Berlin with many tourist attractions has the most higher priced listings as well as most lower priced listings, reflecting its attraction to a diverse range of visitors.
Other districts, like Neukölln for instance, show a clearer tendency towards a more lower price segment, with a dominance of cheaper listings.

Next, we're going to split up the price structure by the number of beds offered. This should gives as a more granular overview of the price differences between districts.

In [ ]:
# We're going to look at appartements with at least one and maximally 8 beds
ber_avg_price = ber_avg_price[(ber_avg_price["beds"]<=8) & (ber_avg_price["beds"]>0)]
ber_avg_price["beds"] = ber_avg_price["beds"].astype(int)

price_per_beds = ber_avg_price.groupby(["neighbourhood_group_cleansed", "beds"])["price"].median().unstack()

In [ ]:
plt.figure(figsize=(8,5))
sns.heatmap(price_per_beds, annot=True, fmt=".0f", cmap="YlGnBu", cbar_kws={'label': 'Median Prize ($)'})
plt.xlabel("Beds")
plt.ylabel("")
plt.title("Median Price by District and Number of Beds", size=14, pad=12)
plt.tight_layout();

For listing with up to 4 beds a similar picture emerges as before, with Berlin-Mitte for example in the higher price category. In contrast to that, for listings with 5 and more beds this clear separation between districts disappears.

### 2. Do visitor numbers and listing prices fluctuate over the year?

There is no direct way to get the number of actual visitors from the data. However, one can use the amount of written reviews as an indication of the actual number of visitors.

In [ ]:
ber_reviews_url = "http://data.insideairbnb.com/germany/be/berlin/2019-07-11/visualisations/reviews.csv"
ber_reviews = pd.read_csv(ber_reviews_url)

ber_reviews["date"] = pd.to_datetime(ber_reviews["date"])
ber_reviews.set_index(ber_reviews["date"], inplace=True)
ber_reviews = ber_reviews[["listing_id"]]

# Selecting the reviews for the last 4 years
ber_reviews = ber_reviews.loc['2015-7-12':'2019-7-11']

# Calculating total reviews per month
# Divided by 4, because we're looking at the average count over a 4 year period
ber_monthly_reviews = ber_reviews.groupby(ber_reviews.index.month).count()/4
ber_monthly_reviews.reset_index(inplace=True)

In [ ]:
# Adding a datetime index to the calendar data
ber_calendar["date"] = pd.to_datetime(ber_calendar["date"])
ber_calendar.set_index(ber_calendar["date"], inplace=True)

# Aggregating monthly data
ber_month_avg = ber_calendar.groupby(ber_calendar.index.month)["price"].mean()
ber_month_avg = ber_month_avg.reset_index()

months = {1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun", 
          7:"Jul", 8:"Aug", 9:"Sep", 10:"Okt", 11:"Nov", 12:"Dez"}

ber_monthly_reviews["date"] = ber_monthly_reviews["date"].replace(months)
ber_month_avg["date"] = ber_month_avg["date"].replace(months)

In [ ]:
fig, ax1 = plt.subplots(figsize=(6, 4))

color = "#0485d1"
ax1.set_xlabel('')
ax1.set_ylabel('Average Monthly Reviews', color=color, size=12)
ax1.plot(ber_monthly_reviews["date"], ber_monthly_reviews["listing_id"], color=color, marker="o", lw=2)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = "#ff474c"
ax2.set_ylabel('Average Monthly Price ($)', color=color, size=12) 
ax2.plot(ber_month_avg["date"], ber_month_avg["price"], color=color, marker="o", lw=2)
ax2.set_ylim(60,70)

plt.title("Monthly Changes in Number of\nReviews and Listing Prices", size=14, pad=12)
fig.tight_layout();

text text text

### 3. How do Airbnb prices in Berlin compare to another large german city, Munich?

In [ ]:
# Loading the calendar data for Munich
mun_calendar_url = "http://data.insideairbnb.com/germany/bv/munich/2019-07-16/data/calendar.csv.gz"
mun_calendar = pd.read_csv(mun_calendar_url)

# Converting the price variable in mun_calendar to numeric
mun_calendar["price"] = pd.to_numeric(mun_calendar["price"].str.replace("$",""), errors="coerce")
mun_calendar = mun_calendar.dropna(subset=["price"])

# Calculating the average price for every listing throughout the whole year
mun_avg_price = mun_calendar.groupby("listing_id")["price"].mean()
mun_avg_price = mun_avg_price.reset_index()

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(ber_avg_price["price"] , color="#0485d1", label="Berlin", kde=False)

ber_mean = ber_avg_price["price"].mean()
plt.axvline(ber_mean, color="#0485d1", linestyle='--', ymax=0.96)
plt.text(69, 7100, "Berlin Average\nListing Price:{:.2f}$".format(ber_mean), fontsize = 11)

sns.distplot(mun_avg_price["price"] , color="#ff474c", label="Munich", kde=False)

mun_mean = mun_avg_price["price"].mean()
plt.axvline(mun_mean, color="#ff474c", linestyle='--', ymax=0.5)
plt.text(113, 3400, "Munich Average\nListing Price:{:.2f}$".format(mun_mean), fontsize = 11)

plt.xlim(0, 300)
plt.xlabel("Listing Price ($)", size=12)
plt.ylabel("Number of Listings", size=12)
plt.title("Distribution of Airbnb Listing Prices\nin Berlin and Munich", size=14, pad=12)
sns.despine()
plt.tight_layout()
plt.legend(frameon=False)
plt.savefig("fig5.png", dpi=100);

The comparison of the price distributions shows that Berlin has more listings in total and more listings in lower price segments (< 50\\$). Munich on the other hand has a higher proportion of highly priced listings. Accordingly the average listing price in Munich is 110.71\\$ compared to 66.54$ in Berlin.

In [ ]:
mun_reviews_url = "http://data.insideairbnb.com/germany/bv/munich/2019-07-16/visualisations/reviews.csv"
mun_reviews = pd.read_csv(mun_reviews_url)

mun_reviews["date"] = pd.to_datetime(mun_reviews["date"])
mun_reviews.set_index(mun_reviews["date"], inplace=True)
mun_reviews = mun_reviews[["listing_id"]]

# Selecting the reviews for the last 4 years
mun_reviews = mun_reviews.loc['2015-7-12':'2019-7-11']

# Calculating total reviews per month
# Divided by 4, because we're looking at the average count over a 4 year period
mun_monthly_reviews = mun_reviews.groupby(mun_reviews.index.month).count()/4
mun_monthly_reviews.reset_index(inplace=True)

mun_monthly_reviews["date"] = mun_monthly_reviews["date"].replace(months)

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(ber_monthly_reviews["date"], ber_monthly_reviews["listing_id"], marker="o", lw=2, color="#0485d1")
plt.plot(mun_monthly_reviews["date"], mun_monthly_reviews["listing_id"], marker="o", lw=2, color="#ff474c")

plt.ylabel("")
plt.text(0,7800, "Berlin", size=12, color="#0485d1")
plt.text(0,2600, "Munich", size=12, color="#ff474c")
plt.title("Average Number of Reviews per Month", size=14, pad=12)
sns.despine();